In [4]:
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np

You have been provided with a dataset of over 350,000 such complaints for 5 common issue types. Your goal is to train a text classification model to identify the issue type based on the consumer complaint narrative. The data can be downloaded from

In [6]:
# Read in data
complaints = pd.read_csv('../data/complaints.csv')

# Standardized column naming
complaints.columns = [x.replace('Consumer ', '').lower().replace(' ', '_') for x in complaints.columns]
complaints['complaint_length'] = complaints['complaint_narrative'].apply(len)

In [8]:
# Explore the count of unique complaints for each issue.
complaints['issue'].value_counts()

Incorrect information on your report    229305
Attempts to collect debt not owed        73163
Communication tactics                    21243
Struggling to pay mortgage               17374
Fraud or scam                            12347
Name: issue, dtype: int64

In [9]:
# Explore some data from the narratives column.
#there appears to be multiple areas where data is redacted.
complaints['complaint_narrative']

0         My name is XXXX XXXX this complaint is not mad...
1         I searched on XXXX for XXXXXXXX XXXX  and was ...
2         I have a particular account that is stating th...
3         I have not supplied proof under the doctrine o...
4         Hello i'm writing regarding account on my cred...
                                ...                        
353427           Collections account I have no knowledge of
353428    Dear CFPB Team, The reason for my complaint is...
353429    FRCA violations : Failing to Follow Debt Dispu...
353430    My Father, a XXXX XXXX  acquired an HECM rever...
353431    I have tried to contact cash app about a fraud...
Name: complaint_narrative, Length: 353432, dtype: object

In [10]:
# Looking at some narratives inside the "Incorrect information" complaint.
seed = 123
for complaint in complaints.loc[complaints['issue'] == 'Incorrect information on your report', 'complaint_narrative'].sample(3, random_state=seed):
    print(complaint)
    print('-----------------------------')

I just reviewed a copy of my Experian credit report and found the following information to be an error. I am a victim of identity theft and did not make these charges. I reported the theft of my identity to the Federal Trade Commission.
-----------------------------
after my legal separation from my husband he started to open credit in my name with no permission I have a legal case against him but can not find him he has ruined my life his name is XXXX XXXX XXXX   remove I have disputed by mail with all three credit bureaus. act # XXXX owing an alleged {$2200.00} XXXX   XXXX  orig creditor XXXX
-----------------------------
Equifax report a collections account " XXXX XXXX XXXX ''. I have no knowledge of this account. I have not been furnished any proof or verifications of this account. I don't have any signed contract agreements with XXXX XXXX XXXX, verbal agreements nor any paperwork associated with them. A proper investigation according to FCRA was not conducted, word of mouth from t

In [13]:
# replacing strings in narrative to remove consecutive x's
complaints['complaint_narrative'] = complaints['complaint_narrative'].str.replace(r'[xX][xX]+','', regex=True)

#Looking at cleaned output that has occurence of only one "x"
complaints.groupby('issue')['complaint_narrative'].apply(lambda x: x.str.extractall(r'(\w+X+\w+)').groupby(0).size().reset_index(name='count'))


0  count
issue                                                        
Attempts to collect debt not owed 0                3Xs      1
                                  1    AMERICANEXPRESS      2
                                  2            ANXIETY      1
                                  3      APPROXIMATELY      3
                                  4             AXCESS      1
...                                                ...    ...
Struggling to pay mortgage        132            TOXIC      5
                                  133            TXHAF     11
                                  134      UNEXCUSABLE      1
                                  135      UNEXPLAINED      1
                                  136          eXPlain      1

[737 rows x 2 columns]